In [1]:
import elo

In [2]:
test = elo.ELO(player_list=['Ryan','Sean','Michael'])

In [3]:
test.EM.show_cache()

{'Michael': 1600, 'Sean': 1600, 'Ryan': 1600}


In [4]:
test.do_competition(winner='Ryan',loser='Sean')
test.EM.show_cache()

{'Michael': 1600, 'Sean': 1584.0, 'Ryan': 1616.0}


In [5]:
test.do_competition(winner='Ryan',loser='Sean')
test.EM.show_cache()

{'Michael': 1600, 'Sean': 1569.4695015289756, 'Ryan': 1630.5304984710244}


In [6]:
test.season_reset()
test.EM.show_cache()

{'Michael': 1600.0, 'Sean': 1584.7347507644877, 'Ryan': 1615.2652492355123}


In [ ]:
test